<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize 
# import k-means from clustering stage
from sklearn.cluster import KMeans


In [2]:
!pip install lxml
!pip install geocoder
import geocoder

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



## 1. Data Source and Cleaning 

Reading tables using pandas from Wikipidia page


In [3]:
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0,keep_default_na=False)
toronto_data = tables[0]
toronto_data.shape

(180, 3)

Ignore cells with a borough that is Not assigned. 

In [4]:
toronto_data.drop(toronto_data.loc[toronto_data['Borough']=='Not assigned'].index, inplace=True)
#toronto_data = toronto_data[toronto_data.Borough != 'Not assigned'] -- alternate code
toronto_data.shape

(103, 3)

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
toronto_data['Neighbourhood'] = np.where(toronto_data['Neighbourhood'] == 'Not assigned', toronto_data['Borough'], toronto_data['Neighbourhood']) # It is not needed since NA to data.
toronto_data[toronto_data['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


More than one neighborhood can exist in one postal code area. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [6]:
toronto_data.groupby(by=['Postal Code','Borough']).agg({'Neighbourhood':', '.join})
print(toronto_data.head())
toronto_data.shape

  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


(103, 3)

### Q1 Result:
### TORONTO DATA SHAPE : (103, 3)

Geographical coordinates of each postal code: 
# geocoder didn't work properly

In [7]:
import geocoder # import geocoder
codes = toronto_data['Postal Code'].tolist()  
codes = codes[:2]
for postal_code in codes:
# initialize your variable to None
    lat_lng_coords = None
    cnt = 20
    print("Code :", postal_code)

# loop until you get the coordinates
    while(lat_lng_coords is None and cnt == 0):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
      cnt -= 1
    if lat_lng_coords is not None:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    print(lat_lng_coords)

Code : M3A
None
Code : M4A
None


Geographical coordinates of each postal code using csv file

In [8]:
geo_cord = pd.read_csv('http://cocl.us/Geospatial_data')
geo_cord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


# Q2

Merging geo cordinates to the toronto data

In [9]:
toronto_data_with_geo = pd.merge(toronto_data,geo_cord,on='Postal Code') #default inner join 
toronto_data_with_geo

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Q3
I followed the same approach as New York data and made my analysis using K-means clustering.

<h3> 1. Explore Neighborhoods in Toronto City </h3>

In [16]:
neighborhoods = toronto_data_with_geo[['Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
neighborhoods.head()
#neighborhoods.shape

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [17]:
neighborhoods.shape

(103, 4)

#### Use geopy library to get the latitude and longitude values of Toronto City.

In [18]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [19]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Toronto. So let's slice the original dataframe and create a new dataframe of the Toronto data.

In [20]:
tor_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
tor_data.head()

print(tor_data.shape)
print(tor_data.loc[0, 'Neighbourhood'])

(39, 4)
Regent Park, Harbourfront


#### Define Foursquare API to get json data

In [22]:
neighborhood_latitude = tor_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = tor_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = tor_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [23]:
CLIENT_ID = 'CROR0TLNBZEMVIFU11M2MWBAWHHWPCBJ1UCTEPWX5UZ1NH0U' # your Foursquare ID
CLIENT_SECRET = 'TPMSNRTQ054CIWNCXJFL2RWD0M2RLCF5NQXUCSA024FJCRRN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
# type your answer here

LIMIT = 50 # limit of number of venues returned by Foursquare API



radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

Your credentails:
CLIENT_ID: CROR0TLNBZEMVIFU11M2MWBAWHHWPCBJ1UCTEPWX5UZ1NH0U
CLIENT_SECRET:TPMSNRTQ054CIWNCXJFL2RWD0M2RLCF5NQXUCSA024FJCRRN


'https://api.foursquare.com/v2/venues/explore?&client_id=CROR0TLNBZEMVIFU11M2MWBAWHHWPCBJ1UCTEPWX5UZ1NH0U&client_secret=TPMSNRTQ054CIWNCXJFL2RWD0M2RLCF5NQXUCSA024FJCRRN&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=50'

In [24]:
results = requests.get(url).json()
#results -- data is long

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
venues

nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues.head()
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980


In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

45 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Toronto

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
tor_venues = getNearbyVenues(names=tor_data['Neighbourhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [30]:
print(tor_venues.shape)
tor_venues.head()



(1190, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [31]:
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,50,50,50,50,50,50
"Brockton, Parkdale Village, Exhibition Place",26,26,26,26,26,26
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,50,50,50,50,50,50
Christie,17,17,17,17,17,17
Church and Wellesley,50,50,50,50,50,50
"Commerce Court, Victoria Hotel",50,50,50,50,50,50
Davisville,34,34,34,34,34,34


In [32]:
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 216 uniques categories.


## 3. Analyze Each Neighborhood

In [34]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
tor_onehot.shape

(1190, 216)

In [37]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped
tor_grouped.shape

(39, 216)

In [38]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.04
2            Beer Bar  0.04
3         Cheese Shop  0.04
4  Seafood Restaurant  0.04


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.12
1            Coffee Shop  0.08
2         Breakfast Spot  0.08
3  Performing Arts Venue  0.08
4                 Bakery  0.08


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                  venue  freq
0    Light Rail Station  0.11
1  Gym / Fitness Center  0.06
2         Auto Workshop  0.06
3            Comic Shop  0.06
4           Pizza Place  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0   Airport Service  0.18
1    Airport Lounge  0.12
2   Harbor / Marina  0.06
3          Boutique  0.06
4  Sculpture Garden  0.06


----Central Bay Stree

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Restaurant,Bakery,Café,Cocktail Bar,Liquor Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Bakery,Climbing Gym,Burrito Place,Nightclub,Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden Center,Skate Park,Brewery,Spa,Burrito Place,Restaurant,Butcher,Recording Studio,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boat or Ferry,Boutique,Rental Car Location,Sculpture Garden,Plane,Harbor / Marina,Coffee Shop,Bar
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Modern European Restaurant,Japanese Restaurant,Juice Bar,Middle Eastern Restaurant


## 4. Cluster Neighborhoods

In [41]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 3, 0, 0, 3, 0, 0, 3, 0], dtype=int32)

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_data
#df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)
tor_merged.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Park,Bakery,Café,Breakfast Spot,Restaurant,Theater,Historic Site,Event Space
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Yoga Studio,College Cafeteria,Smoothie Shop,Beer Bar,Sandwich Place,Café,Portuguese Restaurant,Park,College Auditorium
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Ramen Restaurant,Bookstore,Theater,Clothing Store,Italian Restaurant,Cosmetics Shop,Seafood Restaurant,Spa
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Cosmetics Shop,Coffee Shop,Restaurant,Park,Hotel,Gastropub,Seafood Restaurant,Creperie,Farmers Market
4,East Toronto,The Beaches,43.676357,-79.293031,3,Trail,Pub,Health Food Store,Wine Bar,Cupcake Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

In [45]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Pub,Park,Bakery,Café,Breakfast Spot,Restaurant,Theater,Historic Site,Event Space
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Yoga Studio,College Cafeteria,Smoothie Shop,Beer Bar,Sandwich Place,Café,Portuguese Restaurant,Park,College Auditorium
2,"Garden District, Ryerson",Coffee Shop,Café,Ramen Restaurant,Bookstore,Theater,Clothing Store,Italian Restaurant,Cosmetics Shop,Seafood Restaurant,Spa
3,St. James Town,Café,Cosmetics Shop,Coffee Shop,Restaurant,Park,Hotel,Gastropub,Seafood Restaurant,Creperie,Farmers Market
5,Berczy Park,Coffee Shop,Seafood Restaurant,Cheese Shop,Farmers Market,Beer Bar,Restaurant,Bakery,Café,Cocktail Bar,Liquor Store
6,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop,Burger Joint,Café,Modern European Restaurant,Japanese Restaurant,Juice Bar,Middle Eastern Restaurant
8,"Richmond, Adelaide, King",Coffee Shop,Café,Steakhouse,Concert Hall,American Restaurant,Hotel,Restaurant,Brazilian Restaurant,Plaza,Bookstore
10,"Harbourfront East, Union Station, Toronto Islands",Aquarium,Coffee Shop,Plaza,Brewery,Café,Park,Hotel,Salad Place,IT Services,Dance Studio
11,"Little Portugal, Trinity",Bar,Asian Restaurant,Coffee Shop,Café,Men's Store,Restaurant,Vietnamese Restaurant,Miscellaneous Shop,Mexican Restaurant,Ice Cream Shop
13,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Café,Seafood Restaurant,Restaurant,Hotel,Beer Bar,Japanese Restaurant,Gastropub,Train Station,Plaza


In [46]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",Sushi Restaurant,Park,Jewelry Store,Trail,Wine Bar,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
33,Rosedale,Park,Playground,Trail,Wine Bar,Cupcake Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [47]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Garden,Home Service,Wine Bar,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [48]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,Trail,Pub,Health Food Store,Wine Bar,Cupcake Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
7,Christie,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Italian Restaurant,Baby Store,Diner,Coffee Shop,Nightclub
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Grocery Store,Supermarket,Bank,Bar,Café,Art Gallery,Middle Eastern Restaurant,Music Venue
12,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
14,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Bakery,Climbing Gym,Burrito Place,Nightclub,Restaurant,Italian Restaurant
15,"India Bazaar, The Beaches West",Sandwich Place,Pizza Place,Fast Food Restaurant,Gym,Steakhouse,Ice Cream Shop,Sushi Restaurant,Restaurant,Fish & Chips Shop,Italian Restaurant
22,"High Park, The Junction South",Mexican Restaurant,Café,Thai Restaurant,Flea Market,Fried Chicken Joint,Bar,Bakery,Cajun / Creole Restaurant,Italian Restaurant,Arts & Crafts Store
24,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,History Museum,BBQ Joint,Middle Eastern Restaurant,Indian Restaurant,Pub,Burger Joint,Donut Shop
25,"Parkdale, Roncesvalles",Gift Shop,Breakfast Spot,Italian Restaurant,Coffee Shop,Movie Theater,Cuban Restaurant,Restaurant,Dessert Shop,Dog Run,Bookstore
26,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Sushi Restaurant,Coffee Shop,Italian Restaurant,Café,Gym,Brewery,Seafood Restaurant


In [49]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Swim School,Park,Dim Sum Restaurant,Bus Line,Wine Bar,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


## Analysis Conclusion
    I made a conclusion based on most common venue amoung neighbourhoods with eye-sight. 
    If you can give you inputs on how to make a proper conclusion based on data is much appreciated. 
    
    Cluster 1 : Coffe shops & Restaurants
    Cluster 2 : Park & Trails
    Cluster 3 : Home shopping 
    Cluster 4 : Restaurants & Cafe's
    Cluster 5 : SCommunity